# Deep Learning: Charity Funding Predictor

## Contributor: Sam Espe

### Step 3: Optimizing the Model - Attempt 3

For this attempt, I am increasing the number of nodes in the first layer and decreasing the number of nodes in the hidden layer. I also increased the size of the "Other" bins from attempt 1 to see if that might help.

### Reading in Data; Connecting to Google Drive to save output file

In [2]:
# Import dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [3]:
# Read in charity_data.csv


application_df = pd.read_csv('https://charity-funding-data-set.s3.us-east-2.amazonaws.com/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### Pre-Process the Data


In [5]:
# Split off IS_SUCCESSFUL (target) from the feature variables
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns= "IS_SUCCESSFUL")

In [6]:
# Remove unnecessary columns (EIN & NAME)
X = X.drop(columns= ["EIN", "NAME"])

In [7]:
# Find distribution of unique values of Application Type
X["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Save the value counts series

app_type = X["APPLICATION_TYPE"].value_counts()

In [9]:
# Turn the series into a dictionary

app_dict = app_type.to_dict()

In [10]:
# If the application type has fewer than 100 instances, add it to the list of low-frequency application types.

small_apps = []

for key, value in app_dict.items():
  if int(value) < 200:
    small_apps.append(key)

In [11]:
# If an entry in the Application Type column of the dataframe contains an application type that is in the low-frequency list, replace the value with "Other".

for line in range(len(X)):
    if X.iloc[line, 0] in small_apps:
        X.iloc[line, 0] = "Other"

In [12]:
# Find the distribution of unique values of Classification

X["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [13]:
# Save the value counts of the unique classifications

classes = X["CLASSIFICATION"].value_counts()

In [14]:
# Turn the series into a dictionary

classes_dict = classes.to_dict()

In [15]:
# If the class has fewer than 200 instances, add the class to the list of low-frequency classes.

small_classes = []

for key, value in classes_dict.items():
    if int(value) < 500:
        small_classes.append(key)

In [16]:
# If an entry in the Classification column of the dataframe contains a classification that is in the low-frequency list, replace the value with "Other".

for line in range(len(X)):
    if X.iloc[line, 2] in small_classes:
        X.iloc[line, 2] = "Other"

In [17]:
# One-hot encode categorical variables (Application Type, Classification, Use Case, Organizaton Type, Income Amt, Special Considerations)

X_encoded_df = pd.get_dummies(X)

In [18]:
# The Special Considerations column was encoded twice: once where Y was encoded as 1, and once where N was encoded as 1. I am removing the column where N was encoded as 1 to
# avoid having duplicate data.

X_dummies = X_encoded_df.drop(columns = "SPECIAL_CONSIDERATIONS_N")

In [19]:
# Split data set into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X_encoded_df, y, random_state = 7)

In [20]:
# Scale the data

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [21]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Create the Neural Network

In [31]:
# Create the model

optimization_3_model = tf.keras.models.Sequential()

In [32]:
# Add the input layer

optimization_3_model.add(tf.keras.layers.Dense(units = 200, activation = "relu", input_dim = 44))

In [33]:

# Add the first hidden layer
optimization_3_model.add(tf.keras.layers.Dense(units = 100, activation = "relu"))

In [34]:
# Add the output layer

optimization_3_model.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

In [35]:
optimization_3_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 200)               9000      
                                                                 
 dense_5 (Dense)             (None, 100)               20100     
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 29,201
Trainable params: 29,201
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model

optimization_3_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [37]:
# Fit the model to the training set

fit_model = optimization_3_model.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5697 - accuracy: 0.7241
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5553 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7301
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7314
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5486 - accuracy: 0.7330
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7331
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7330
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5460 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5448 - accuracy: 0.7347
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5446 - accura

In [38]:
# Evaluate the model using the testing data

model_loss, model_accuracy = optimization_3_model.evaluate(X_test_scaled, y_test, verbose = 2)

print(f'Loss: {round(model_loss * 100, 2)}%, Accuracy: {round(model_accuracy * 100, 2)}%')

268/268 - 1s - loss: 0.5870 - accuracy: 0.7268 - 511ms/epoch - 2ms/step
Loss: 58.7%, Accuracy: 72.68%


In [39]:
# Save and export results as HDF5 file

optimization_3_model.save("Alphabet_Soup_Charity_Optimization_3.h5")

The original model had an accuracy of 72.79% and a loss of 58.79%. This attempt has an accuracy of 72.68% and a loss of 58.70%. The models performed about the same.